<a href="https://colab.research.google.com/github/yathishlax/seq2seq-model/blob/master/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Important libraries

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data

In [7]:
batch_size = 64 # batch size for training
epochs = 100 #numbers of epochs for training
latent_dim = 256 #latent domensionality of encoding space
num_samples = 10000 #number of samples to train on



## Path to datafile

In [5]:
datapath = 'fra.txt'

## Vectorize the data

In [8]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(datapath, 'r', encoding = 'utf-8') as f:
  lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines)-1)]:
  input_text, target_text, _ = line.split('\t')
  #we use tab as 'short requence' character
  #for the targets and '\n' as "end sequence" character
  target_text =  '\t' +target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)



## lets see the classification

In [16]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(len(txt)for txt in input_texts)
max_decoder_seq_length = max(len(txt)for txt in target_texts)

In [17]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


## Lets assign the token for each and every chararcter

In [18]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [20]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  '%': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '0': 9,
  '1': 10,
  '2': 11,
  '3': 12,
  '5': 13,
  '6': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A'

## Creating 3D numpy arrays for input and output

In [22]:
import numpy as np

In [23]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')

## One hot representation 

In [33]:
for i, (input_text, target_text), in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t:, input_token_index[char]]=1.
    encoder_input_data[i, t+1:, input_token_index[' ']]=1.
    for t, char in enumerate(target_text):
      #decoder_target_data is ahead of decoder_input_data by one timestamp
      decoder_input_data[i, t, target_token_index[char]]=1.
      if t >0:
        #decoder_target_data will be ahead by one timestep
        #and will not include start character
        decoder_target_data[i, t - 1, target_token_index[char]]= 1.
      decoder_input_data[i, t + 1:, target_token_index[' ']]= 1. 
      decoder_target_data[i, t:, target_token_index[' ']]= 1. 

In [34]:
encoder_input_data[0].shape

(16, 71)

## Applying LSTM

In [37]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [39]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [41]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [42]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 73s 9ms/step - loss: 1.3835 - val_loss: 1.3751
Epoch 2/100
8000/8000 [==============================] - 71s 9ms/step - loss: 1.1294 - val_loss: 1.2125
Epoch 3/100
8000/8000 [==============================] - 74s 9ms/step - loss: 1.0005 - val_loss: 1.0900
Epoch 4/100
8000/8000 [==============================] - 71s 9ms/step - loss: 0.9391 - val_loss: 1.0973
Epoch 5/100
8000/8000 [==============================] - 71s 9ms/step - loss: 0.8973 - val_loss: 1.0110
Epoch 6/100
8000/8000 [==============================] - 73s 9ms/step - loss: 0.8696 - val_loss: 0.9892
Epoch 7/100
8000/8000 [==============================] - 74s 9ms/step - loss: 0.8456 - val_loss: 0.9747
Epoch 8/100
8000/8000 [==============================] - 74s 9ms/step - loss: 0.8258 - val_loss: 0.9559
Epoch 9/100
8000/8000 [==============================] - 73s 9ms/step - loss: 0.8074 - val_loss: 0.9450
Epoch 10/100
800

In [44]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]